In [31]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import sys
import re

In [2]:
path = Path(os.getcwd())
base_dir = path.parent.parent

In [3]:
#Windows format
data_in = os.path.join(str(base_dir) , "da_data_repo\\bisnode-firms\\clean\\")
data_out = os.path.join(str(base_dir) , "da_data_repo\\bisnode-firms\\")
output = os.path.join(data_out,'output')
func = os.path.join(str(base_dir) ,   "da_case_studies\\ch00-tech-prep\\")

In [4]:
current_csv_path = os.path.join(data_out,"bisnode_firms_clean.csv")
data = pd.read_csv(current_csv_path)

In [11]:
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19036 entries, 0 to 19035
Data columns (total 118 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   year                            int64  
 1   comp_id                         float64
 2   begin                           object 
 3   end                             object 
 4   amort                           float64
 5   curr_assets                     float64
 6   curr_liab                       float64
 7   extra_exp                       float64
 8   extra_inc                       float64
 9   extra_profit_loss               float64
 10  fixed_assets                    float64
 11  inc_bef_tax                     float64
 12  intang_assets                   float64
 13  inventories                     float64
 14  liq_assets                      float64
 15  material_exp                    float64
 16  personnel_exp                   float64
 17  profit_loss_year              

In [19]:
rawvars = ["curr_assets", "curr_liab", "extra_exp", "extra_inc", "extra_profit_loss", "fixed_assets",
              "inc_bef_tax", "intang_assets", "inventories", "liq_assets", "material_exp", "personnel_exp",
              "profit_loss_year", "sales", "share_eq", "subscribed_cap"]
qualityvars = ["balsheet_flag", "balsheet_length", "balsheet_notfullyear"]
engvar = ["total_assets_bs", "fixed_assets_bs", "liq_assets_bs", "curr_assets_bs",
            "share_eq_bs", "subscribed_cap_bs", "intang_assets_bs", "extra_exp_pl",
            "extra_inc_pl", "extra_profit_loss_pl", "inc_bef_tax_pl", "inventories_pl",
            "material_exp_pl", "profit_loss_year_pl", "personnel_exp_pl"]
engvar2 = ["extra_profit_loss_pl_quad", "inc_bef_tax_pl_quad",
             "profit_loss_year_pl_quad", "share_eq_bs_quad"]


d1 =  ["d1_sales_mil_log_mod", "d1_sales_mil_log_mod_sq",
         "flag_low_d1_sales_mil_log", "flag_high_d1_sales_mil_log"]
hr = ["female", "ceo_age", "flag_high_ceo_age", "flag_low_ceo_age",
        "flag_miss_ceo_age", "ceo_count", "labor_avg_mod",
        "flag_miss_labor_avg", "foreign_management"]
firm = ["age", "age2", "new", "ind2_cat", "m_region_loc", "urban_m"]

# interactions for logit, LASSO
interactions1 = ["ind2_cat*age", "ind2_cat*age2",
                   "ind2_cat*d1_sales_mil_log_mod", "ind2_cat*sales_mil_log",
                   "ind2_cat*ceo_age", "ind2_cat*foreign_management",
                   "ind2_cat*female",   "ind2_cat*urban_m", "ind2_cat*labor_avg_mod"]
interactions2 = ["sales_mil_log*age", "sales_mil_log*female",
                   "sales_mil_log*profit_loss_year_pl", "sales_mil_log*foreign_management"]

X1 = ["sales_mil_log", "sales_mil_log_sq", "d1_sales_mil_log_mod", "profit_loss_year_pl", "ind2_cat"]
X2 = ["sales_mil_log", "sales_mil_log_sq", "d1_sales_mil_log_mod", "profit_loss_year_pl", "fixed_assets_bs",
      "share_eq_bs","curr_liab_bs ",   "curr_liab_bs_flag_high ", "curr_liab_bs_flag_error",  "age",
      "foreign_management" , "ind2_cat"]
X3 = ["sales_mil_log", "sales_mil_log_sq", firm, engvar,                   d1]
X4 = ["sales_mil_log", "sales_mil_log_sq", firm, engvar, engvar2, engvar3, d1, hr, qualityvars]
X5 = ["sales_mil_log", "sales_mil_log_sq", firm, engvar, engvar2, engvar3, d1, hr, qualityvars, interactions1, interactions2]


NameError: name 'engvar3' is not defined

In [30]:
for col in data.columns if ('flag_low') in col:
    
flag_columns

['extra_profit_loss_pl_flag_low',
 'inc_bef_tax_pl_flag_low',
 'profit_loss_year_pl_flag_low',
 'share_eq_bs_flag_low',
 'flag_low_ceo_age',
 'flag_low_d1_sales_mil_log']

In [ ]:
 "profit_loss_year_pl_quad", "share_eq_bs_quad")
engvar3 <- c(grep("*flag_low$", names(data), value = TRUE),
             grep("*flag_high$", names(data), value = TRUE),
             grep("*flag_error$", names(data), value = TRUE),
             grep("*flag_zero$", names(data), value = TRUE))